<a href="https://colab.research.google.com/github/asoumabasouma/Big_data/blob/main/AndroidSparkAitBensaidAsmae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyspark

In [27]:
from pyspark import SparkContext

# Arrêter le SparkContext existant si besoin
try:
    sc.stop()
except:
    pass

sc = SparkContext(appName='TriLogsDistribué')

# Lire ton fichier de logs
android_RDD = sc.textFile("/content/Android.log")



In [28]:
#extraire l’heure et la minute depuis une ligne de log Android.
import re
# Capture HH:MM
pattern = re.compile(r"\d{2}-\d{2}\s(\d{2}:\d{2})")

In [29]:
# Nettoyage : garder seulement les lignes valides
valid_RDD = android_RDD.filter(lambda line: pattern.search(line) is not None)


In [30]:
#compter par minute
minute_RDD = valid_RDD.map(
    lambda line: (pattern.search(line).group(1), 1)
)

count_per_minute = minute_RDD.reduceByKey(lambda a, b: a + b)


In [31]:
# Comptage par heure
count_per_hour = minute_RDD.map(
    lambda x: (x[0][:2], x[1])
).reduceByKey(lambda a, b: a + b)


In [34]:
#extraire minute + level
import re

pattern = re.compile(
    r"\d{2}-\d{2}\s(\d{2}:\d{2}):\d{2}\.\d+\s+\d+\s+\d+\s+([VDIWE])\s"
)

In [35]:
#comptage minute + level
minute_level_RDD = valid_RDD.map(
    lambda line: (
        (pattern.search(line).group(1),   # HH:MM
         pattern.search(line).group(2)),  # Level
        1
    )
)

count_minute_level = minute_level_RDD.reduceByKey(lambda a, b: a + b)


In [36]:
#comptage heure+level
hour_level_RDD = minute_level_RDD.map(
    lambda x: ((x[0][0][:2], x[0][1]), x[1])
)

count_hour_level = hour_level_RDD.reduceByKey(lambda a, b: a + b)


In [40]:
print("=== Messages par minute ===")
for x in count_per_minute.collect():
    print(x)

=== Messages par minute ===
('19:34', 3799)
('19:42', 2207)
('19:43', 164)
('19:52', 311)
('19:55', 3628)
('19:56', 6351)
('19:59', 4455)
('20:01', 3733)
('20:07', 204)
('20:10', 213)
('20:12', 6266)
('20:15', 1888)
('20:18', 4047)
('20:25', 2829)
('20:26', 1427)
('20:31', 12140)
('20:36', 2247)
('20:38', 1543)
('20:45', 1721)
('20:46', 1723)
('20:49', 2391)
('20:51', 2961)
('20:56', 1428)
('20:57', 1217)
('21:06', 1443)
('21:07', 1499)
('21:12', 1343)
('21:18', 1310)
('21:27', 10029)
('21:29', 1232)
('21:30', 1367)
('21:32', 13732)
('21:35', 1130)
('21:36', 1651)
('21:38', 1313)
('21:39', 1316)
('21:40', 1610)
('21:47', 7571)
('21:53', 8894)
('21:59', 3338)
('22:17', 244)
('22:23', 514)
('22:29', 5177)
('22:31', 3160)
('22:32', 1291)
('22:39', 1310)
('22:40', 842)
('22:47', 713)
('22:50', 675)
('22:54', 1289)
('22:55', 1342)
('22:57', 1092)
('23:00', 2463)
('23:10', 3547)
('23:26', 4881)
('23:27', 1894)
('23:28', 2842)
('23:30', 2712)
('23:34', 3740)
('23:43', 1356)
('23:44', 1188)
('

In [41]:
print("\n=== Messages par heure ===")
for x in count_per_hour.collect():
    print(x)


=== Messages par heure ===
('21', 198153)
('22', 96650)
('01', 3998)
('02', 2436)
('09', 58555)
('10', 63337)
('20', 138489)
('23', 134333)
('05', 2010)
('06', 2366)
('12', 23303)
('00', 48236)
('03', 2400)
('04', 2243)
('07', 68650)
('13', 7824)
('14', 6343)
('19', 77059)
('08', 96895)
('11', 17530)


In [42]:
print("=== Messages par minute et level ===")
for x in count_minute_level.collect():
    print(x)

=== Messages par minute et level ===
(('19:31', 'V'), 341)
(('19:32', 'V'), 147)
(('19:33', 'V'), 259)
(('19:34', 'I'), 1640)
(('19:34', 'E'), 329)
(('19:35', 'W'), 227)
(('19:36', 'D'), 323)
(('19:37', 'W'), 15)
(('19:40', 'W'), 239)
(('19:41', 'D'), 29)
(('19:42', 'E'), 337)
(('19:42', 'I'), 1204)
(('19:43', 'I'), 41)
(('19:43', 'E'), 38)
(('19:44', 'W'), 4)
(('19:45', 'D'), 111)
(('19:47', 'D'), 35)
(('19:48', 'V'), 3)
(('19:50', 'D'), 48)
(('19:51', 'W'), 15)
(('19:52', 'I'), 147)
(('19:52', 'E'), 42)
(('19:54', 'D'), 73)
(('19:55', 'I'), 1852)
(('19:55', 'E'), 543)
(('19:56', 'I'), 2376)
(('19:56', 'E'), 442)
(('19:57', 'D'), 4636)
(('19:58', 'D'), 5368)
(('19:59', 'E'), 880)
(('19:59', 'I'), 1377)
(('20:00', 'D'), 1184)
(('20:01', 'E'), 220)
(('20:01', 'I'), 2159)
(('20:02', 'D'), 206)
(('20:03', 'D'), 212)
(('20:05', 'V'), 25)
(('20:06', 'V'), 12)
(('20:07', 'I'), 60)
(('20:07', 'E'), 32)
(('20:10', 'I'), 44)
(('20:10', 'E'), 137)
(('20:11', 'V'), 135)
(('20:12', 'I'), 3795)
(('

In [39]:
print("\n=== Messages par heure et level ===")
for x in count_hour_level.collect():
    print(x)


=== Messages par heure et level ===
(('19', 'W'), 3280)
(('20', 'V'), 5479)
(('21', 'I'), 108394)
(('21', 'E'), 16254)
(('22', 'I'), 49785)
(('22', 'E'), 11419)
(('23', 'V'), 3997)
(('00', 'D'), 16539)
(('01', 'E'), 1197)
(('01', 'I'), 1404)
(('02', 'E'), 383)
(('02', 'I'), 1454)
(('03', 'D'), 679)
(('04', 'D'), 540)
(('05', 'V'), 2)
(('06', 'V'), 12)
(('07', 'D'), 22285)
(('08', 'W'), 3456)
(('09', 'I'), 30941)
(('09', 'E'), 6879)
(('10', 'I'), 27282)
(('10', 'E'), 7359)
(('11', 'W'), 1073)
(('12', 'V'), 638)
(('13', 'D'), 2353)
(('14', 'D'), 1444)
(('19', 'D'), 32534)
(('20', 'I'), 72732)
(('20', 'E'), 10727)
(('21', 'V'), 7044)
(('22', 'V'), 2543)
(('23', 'I'), 75058)
(('23', 'E'), 11745)
(('00', 'W'), 1819)
(('01', 'V'), 37)
(('02', 'V'), 20)
(('03', 'W'), 123)
(('04', 'W'), 93)
(('05', 'E'), 366)
(('05', 'I'), 924)
(('06', 'E'), 532)
(('06', 'I'), 1085)
(('07', 'W'), 2833)
(('08', 'D'), 34806)
(('09', 'V'), 1227)
(('10', 'V'), 2447)
(('11', 'D'), 4356)
(('12', 'I'), 11276)
(('12'